<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/Copy_of_tsclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Part (Python):

In [1]:
import pandas as pd
import numpy as np
from os.path import join
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Data
dfof: d_awake, d_sleep \\
cosine similarity: ts_dist_awake, ts_dist_sleep



In [2]:
summary_new = pd.read_csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/summary_sleep.csv")
data_path = '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/5HT2afl05b_1/7_12_SD2'
file_name = 'dfof_mc.csv'
dfof_data = pd.read_csv(join(data_path, file_name))
dfof_data.set_index('roi_label', drop=True, inplace=True)

In [25]:
def process_dfof_mc(dfof, summary_new):
    n_roi = dfof.shape[0]
    n_time = dfof.shape[1]
    awake = []
    sleep = []
    for j in range(0, 20, 2):
        awake.append(summary_new['end'][j])
        sleep.append(summary_new['end'][j + 1])
    d_awake = dfof.iloc[:, awake]
    d_sleep = dfof.iloc[:, sleep]
    return {'d_awake': d_awake, 'd_sleep': d_sleep}

In [26]:
result = process_dfof_mc(dfof_data, summary_new)

In [6]:
pip install rpy2

In [22]:
%load_ext rpy2.ipython

In [23]:
%%R
process_dfof_mc = function(dfof, summary_new) {
    n_roi = dim(dfof)[1]
    n_time = dim(dfof)[2]
    d = dfof[, 2:n_time]
    awake = c()
    sleep = c()
    for (j in seq(1, 20, 2)) {
        awake = c(awake, summary_new$end[j])
        sleep = c(sleep, summary_new$end[j + 1])
        }
    d_awake = d[, awake] d_sleep = d[, sleep]
    return(list(d_awake = d_awake, d_sleep = d_sleep)) }

result = process_dfof_mc(dfof_data, summary_new)

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [ ]:
import rpy2.robjects as ro
# Assuming you have already obtained the 'result' ListVector
d_awake = result.rx2('d_awake')
d_sleep = result.rx2('d_sleep')

NameError: name 'result' is not defined

In [ ]:
d_awake

1,2,3,...,32298,32299,32300
0.287884,0.248868,0.253810,...,0.170616,0.147643,0.053788
0.201098,0.078216,0.218820,,0.053857,0.232339,0.197037
0.773431,0.699030,0.526918,,0.085772,-0.009612,0.127851
0.112303,0.146420,0.084935,,-0.063514,0.022450,0.061555
...,...,...,,...,...,...
0.102996,0.218701,-0.046223,,0.057780,-0.004482,0.168040
0.000365,0.217156,0.067607,,0.139667,0.146817,0.216039
0.462463,0.189832,0.260967,,0.039161,-0.001700,-0.010567
0.390402,0.083989,0.431694,,0.134920,0.102995,-0.099601


In [ ]:
from scipy.spatial.distance import cosine

X = np.array(d_awake)
n = X.shape[0]
dist = np.empty((n, n))

for i in range(n - 1):
    for j in range(i + 1, n):
        dist[i, j] = 1 - cosine(X[i, :], X[j, :])

min_val = np.min(dist)
max_val = np.max(dist)
ts_dist_awake = (dist - min_val) / (max_val - min_val)

ts_dist_awake.to_csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_awake.csv', index=False)

X = np.array(d_sleep)
n = X.shape[0]
dist = np.empty((n, n))

for i in range(n - 1):
    for j in range(i + 1, n):
        dist[i, j] = 1 - cosine(X[i, :], X[j, :])

min_val = np.min(dist)
max_val = np.max(dist)
ts_dist_sleep = (dist - min_val) / (max_val - min_val)
ts_dist_sleep.to_csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_sleep.csv', index=False)

KeyboardInterrupt: 

Get significant cell:

In [ ]:
import sys
from os.path import join
import matplotlib.pyplot as plt
sys.path.append('/content/sleep/notebooks/neuropil_analysis/')
import neuropil as npil
sys.path.append("/content/sleep/src/")
from eeg_class import eegData
from behavior_class import behaviorData

#Loading sleep and other behavior data (velocity)
sleep_data = pd.read_csv(join(data_path, 'velo_eeg.csv'))
#Determining awake mobile and imobile periods
sleep_data['mobile_immobile'] = behaviorData.define_immobility(
        sleep_data["filtered velo"])
#Creating brain state filters for filtering the datasets
brain_states =  eegData.brain_state_filter(
        velo_eeg_df=sleep_data,
        states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
    )
nrem_mask = brain_states['NREM']
awake_mask = brain_states['awake_immobile']
nrem_intervals = dfof_data.loc[:, nrem_mask.values]
awake_intervals = dfof_data.loc[:, awake_mask.values]
#Calculating significant cells
significant_cells_nrem_vs_awake = npil.statistics.significance_calc(nrem_intervals.to_numpy(),
                                                      awake_intervals.to_numpy())
significant_cells_awake_vs_nrem = npil.statistics.significance_calc(awake_intervals.to_numpy(),
                                                      nrem_intervals.to_numpy())
sig_cell_mask = np.zeros(dfof_data.shape[0], dtype=bool)
sig_cell_mask[significant_cells_nrem_vs_awake] = True
significant_cell_data = dfof_data[sig_cell_mask]
significant_cell_data

ModuleNotFoundError: No module named 'neuropil'

Second Part (R):

In [ ]:
ts_dist_awake = read.csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/ts_dist_awake.csv")
ts_dist_sleep = read.csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_sleep.csv')

net.epsilon.create <- function(dist, epsilon) {
    n = matrix(0, ncol(dist), nrow(dist))
    n[dist < epsilon] = 1;
    return(graph.adjacency(n, mode="undirected", diag=F));
}

In [ ]:
eps_seq = seq(0.01,0.99,0.01)
ddd = function(ts_dist){
modu = c()
n_hubs = c()
n_edges = c()
for (i in 1:length(eps_seq)){
net1 = net.epsilon.create(dist = ts_dist, epsilon = eps_seq[i])
c1 = cluster_fast_greedy(net1)
hub_score1 = hub.score(net1)
n_hubs[i] = length(which(hub_score1[["vector"]]>0.9))
modu[i] = modularity(c1)
n_edges[i] = gsize(net1)
}
return(list(n_hubs,n_edges,modu))
}

tt = tibble(
  eps = rep(eps_seq,2),
  state = c(rep("aw",length(eps_seq)),rep("sl",length(eps_seq))),
  n_hub = c(ddd(ts_dist_awake)[[1]],ddd(ts_dist_sleep)[[1]]),
  n_edge = c(ddd(ts_dist_awake)[[2]],ddd(ts_dist_sleep)[[2]]),
  modu = c(ddd(ts_dist_awake)[[3]],ddd(ts_dist_sleep)[[3]])
)

tt %>%
  ggplot(aes(x=eps,y=modu,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=eps,y=n_hub,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=eps,y=n_edge,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=n_edge,y=n_hub,colour=state))+
  geom_point(size=0.8)+
  geom_line()

## significant sleep Gergo
tibble(
  n = 1:length(rownames(roi_cluster)),
  roi = rownames(roi_cluster)) %>%
  filter(roi %in% sig_cell) %>% view


In [ ]:
ts_dist = ts_dist_sleep
eps = 0.2
net = net.epsilon.create(dist = ts_dist, epsilon = eps)
c = cluster_fast_greedy(net)
membership(c)
plot(net, vertex.color=membership(c), layout=layout_with_fr(net))
#hub score
hub_score <- hub.score(net)
which(hub_score[["vector"]]>=0.9)